# 0. 导入相关包

In [1]:
import json
from elasticsearch7 import Elasticsearch

# 1. 使用Python连接ES

In [2]:
es = Elasticsearch()

+ 检测是否连接成功

In [3]:
print(es.ping())

True


In [48]:
# mysql读取数据
def get_rawdata():
    import mysql.connector
    config = {
        'host' : 'rm-uf6q2j49y3xo8ny5hno.mysql.rds.aliyuncs.com',
        'user' : 'music_root',
        'password' : 'DIANgongdao123root',
        'port' : 3306,
        'database' : 'netease',
        'charset' : 'utf8'
    }
    cnn = mysql.connector.connect(**config) # 建立MySQL连接
    cursor = cnn.cursor() # 获得游标
    cursor.execute('SELECT * FROM taglist')
    result = cursor.fetchall()
    # print(result)
    cursor.close() # 关闭游标
    cnn.close() # 关闭连接
    return result

In [49]:
result = get_rawdata()
# f = open('log.txt','w',encoding="utf-8")
# for i in range(10000):
#     for j in range(6):
#         f.write(str(result[i][j]))
# f.close()
print(result)

[('2058369193', 'https://music.163.com/playlist?id=2058369193', '华语伤感丨泪流不止，只因想你', 'ALLIN_MUSIC', '210391', '2243', '18', '5', '华语|伤感|流行', '介绍：\n如果有一天，我忘了全世界，但我一定会记得你。我会在某一个黄昏，像往常一样，走到你面前，告诉你，爱可以穿越时光的考验，在记忆中盛开一世。\n', '60', '在意这件事|唯一|一吻之间|我怀念的|忘了去记得|最初的温柔|一个人失忆|就这样|那些年|没关系'), ('2190978686', 'https://music.163.com/playlist?id=2190978686', '乐队｜一次有预谋的初次相遇', '格林岛屿', '1043984', '14691', '180', '28', '流行|华语', '介绍：\n从明天起做一个幸福的人，喂马，劈柴周游世界，从明天起，关心粮食和蔬菜，我有一所房子面朝大海，春暖花开。——海子\n\n不妨不妨，来日方长\n人间美好，日落和你......\n', '68', '一次有预谋的初次相遇|夏日漱石 (Summer Cozy Rock)|夏 天 ， 最 后 一 次 恋 爱|给你一瓶魔法药水|别在Livehouse哭泣|海浪|艾蜜莉|你不爱我我就爱别人|有暖气 (You Nuan Chi)|西湖'), ('2217610700', 'https://music.163.com/playlist?id=2217610700', '［戏腔］戏子多秋 可怜一处情深旧', '一许桃花醉如烟', '43751408', '620000', '4304', '1125', '华语|古风', '介绍：\n终于为那一身江南烟雨覆了天下，容华谢后，不过一场，山河永寂。\n\n千秋功名，一世葬你，玲珑社稷，可笑却无君王命。\n\n凤凰台上凤凰游，负约而去，一夜苦等，从此江南江北，万里哀哭。\n\n嗟叹红颜泪、英雄殁，人世苦多。山河永寂、怎堪欢颜。\n\n风华是一指流砂，苍老是一段年华。\n\n夜雨染成天水碧。有些人不需要姿态，也能成就一场惊鸿。\n\n你要记得，紫檀未灭，我亦未去。\n\n谁在岁月里长长叹息。\n\n汉霄苍茫，牵住繁华哀伤，弯眉间，命中注定

# 2. 索引操作

## 2.1 新建索引

In [12]:
mystop = {#以“|”为分隔
  "settings": {
    "analysis": {
      "analyzer": {
        "my_stop_analyzer": {
          "type": "stop",
          "stopwords": ["|"]
        }
      }
    }
  }
}

### 2.1.1 定义mapping

In [39]:
mappings = {
    "mappings" : {
      "properties" : {
        "album_id" : {
            "type" : "long",
        },
        "url" : {
          "type" : "text",
        },
        "name" : {
          "type" : "text",
        },
        "author" : {
            "type" : "text",
        },
        "views" : {
            "type" : "long",
        },
        "subscription" : {
            "type" : "long",
        },
        "label" : {
          "type" : "text",
        },
        "description" : {
          "type" : "text",
        },
        "amount" : {
          "type" : "long",
        },
        "nameofmusic" : {
          "type" : "text",
        }
      }
    }
}

### 2.1.2 依据mapping创建索引

In [51]:
res = es.indices.create(index ='index_music',body =mappings)

C:\Users\BR\AppData\Local\Temp/ipykernel_12284/2365663305.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.indices.create(index ='index_music',body =mappings)


## 2.2 删除索引

In [50]:
es.indices.delete("index_music")

C:\Users\BR\AppData\Local\Temp/ipykernel_12284/2160321965.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.delete("index_music")


{'acknowledged': True}

# 3. 文档操作

## 3.1 插入文档

In [52]:
for i in range(125):
    es.index(index="index_music", id=i,body={"album_id":result[i][0],"url":result[i][1], "name":result[i][2],  "author":result[i][3],"views":result[i][4],"subscription":result[i][5],"label":result[i][8],"description":result[i][9],"amount":result[i][10],"nameofmusic":result[i][11]})

C:\Users\BR\AppData\Local\Temp/ipykernel_12284/2064800677.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index="index_music", id=i,body={"album_id":result[i][0],"url":result[i][1], "name":result[i][2],  "author":result[i][3],"views":result[i][4],"subscription":result[i][5],"label":result[i][8],"description":result[i][9],"amount":result[i][10],"nameofmusic":result[i][11]})


## 3.2 查询文档

In [54]:
es.get(index="index_music", id=0, ignore=404)

{'_index': 'index_music',
 '_type': '_doc',
 '_id': '0',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'album_id': '2058369193',
  'url': 'https://music.163.com/playlist?id=2058369193',
  'name': '华语伤感丨泪流不止，只因想你',
  'author': 'ALLIN_MUSIC',
  'views': '210391',
  'subscription': '2243',
  'label': '华语|伤感|流行',
  'description': '介绍：\n如果有一天，我忘了全世界，但我一定会记得你。我会在某一个黄昏，像往常一样，走到你面前，告诉你，爱可以穿越时光的考验，在记忆中盛开一世。\n',
  'amount': '60',
  'nameofmusic': '在意这件事|唯一|一吻之间|我怀念的|忘了去记得|最初的温柔|一个人失忆|就这样|那些年|没关系'}}

In [55]:
# json格式化显示函数
def json_print(string):
    print(json.dumps(string, sort_keys=True, indent=4, separators=(',', ':')))
    return

In [56]:
res = es.get(index="index_music", id=0, ignore=404)
json_print(res)

{
    "_id":"0",
    "_index":"index_music",
    "_primary_term":1,
    "_seq_no":0,
    "_source":{
        "album_id":"2058369193",
        "amount":"60",
        "author":"ALLIN_MUSIC",
        "description":"\u4ecb\u7ecd\uff1a\n\u5982\u679c\u6709\u4e00\u5929\uff0c\u6211\u5fd8\u4e86\u5168\u4e16\u754c\uff0c\u4f46\u6211\u4e00\u5b9a\u4f1a\u8bb0\u5f97\u4f60\u3002\u6211\u4f1a\u5728\u67d0\u4e00\u4e2a\u9ec4\u660f\uff0c\u50cf\u5f80\u5e38\u4e00\u6837\uff0c\u8d70\u5230\u4f60\u9762\u524d\uff0c\u544a\u8bc9\u4f60\uff0c\u7231\u53ef\u4ee5\u7a7f\u8d8a\u65f6\u5149\u7684\u8003\u9a8c\uff0c\u5728\u8bb0\u5fc6\u4e2d\u76db\u5f00\u4e00\u4e16\u3002\n",
        "label":"\u534e\u8bed|\u4f24\u611f|\u6d41\u884c",
        "name":"\u534e\u8bed\u4f24\u611f\u4e28\u6cea\u6d41\u4e0d\u6b62\uff0c\u53ea\u56e0\u60f3\u4f60",
        "nameofmusic":"\u5728\u610f\u8fd9\u4ef6\u4e8b|\u552f\u4e00|\u4e00\u543b\u4e4b\u95f4|\u6211\u6000\u5ff5\u7684|\u5fd8\u4e86\u53bb\u8bb0\u5f97|\u6700\u521d\u7684\u6e29\u67d4|\u4e00\u4e2a\u4eba\u5

## 3.3 修改（更新）文档

In [98]:
# for i in range(10000):
#     authors = ""
#     keywords = ""
#     for author in result[i][3]:
#         authors += author
#         authors += " "
#     for keyword in result[i][4]:
#         keywords += keyword
#         keywords += " "
#     res = es.update(index="index_paper", id=i, body={"doc":{"authors": authors,"keywords": keywords}})
# json_print(res)

C:\Users\BR\AppData\Local\Temp/ipykernel_8780/896306200.py:10: DeprecationWarning: The 'body' parameter is deprecated for the 'update' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.update(index="index_paper", id=i, body={"doc":{"authors": authors,"keywords": keywords}})


In [30]:
res = es.get(index="index_test", id=0, ignore=404)
json_print(res)

{
    "_id":"0",
    "_index":"index_test",
    "_primary_term":1,
    "_seq_no":1,
    "_source":{
        "age":22,
        "hobby":"rap",
        "name":"Tom"
    },
    "_type":"_doc",
    "_version":2,
    "found":true
}


## 3.4 删除文档

In [31]:
es.delete(index="index_test", id=0)

{'_index': 'index_test',
 '_type': '_doc',
 '_id': '0',
 '_version': 3,
 'result': 'deleted',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}

In [32]:
res = es.get(index="index_test", id=0, ignore=404)
json_print(res)

{
    "_id":"0",
    "_index":"index_test",
    "_type":"_doc",
    "found":false
}


## 3.5 bulk批量操作

In [33]:
doc = [
     {"create": {"_index": "index_test", '_id': 0 }},
     {'name': 'Tom', 'age': 20, 'hobby': 'football rap'},
     {"create": {"_index": "index_test",  '_id': 1 }},
     {'name': 'Jerry', 'age': 29, 'hobby': 'basketball sing football'},
     {"create": {"_index": "index_test",  '_id': 2}},
     {'name': 'Bob', 'age': 12, 'hobby': 'football dance'},
     {"create": {"_index": "index_test",  '_id': 3}},
     {'name': 'Amy', 'age': 24, 'hobby': 'sing dance'},
     {"create": {"_index": "index_test",  '_id': 4}},
     {'name': 'Jack', 'age': 19, 'hobby': 'sing'},
 ]

In [34]:
res = es.bulk(index='index_test',  body=doc)
json_print(res)

{
    "errors":false,
    "items":[
        {
            "create":{
                "_id":"0",
                "_index":"index_test",
                "_primary_term":1,
                "_seq_no":3,
                "_shards":{
                    "failed":0,
                    "successful":1,
                    "total":2
                },
                "_type":"_doc",
                "_version":4,
                "result":"created",
                "status":201
            }
        },
        {
            "create":{
                "_id":"1",
                "_index":"index_test",
                "_primary_term":1,
                "_seq_no":4,
                "_shards":{
                    "failed":0,
                    "successful":1,
                    "total":2
                },
                "_type":"_doc",
                "_version":1,
                "result":"created",
                "status":201
            }
        },
        {
            "create":{
           

# 4. 搜索操作

## 仅以query DSL为例，其余类型

In [59]:
query = {
    "query": {
    "match": {
      "name": "说唱"
    }
  },
    "highlight":{
        "fields":{
            "name":{}
        }
    }
}

In [60]:
result = es.search(index="index_music", body=query)
print(result)

{'took': 9, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 17, 'relation': 'eq'}, 'max_score': 5.89508, 'hits': [{'_index': 'index_music', '_type': '_doc', '_id': '80', '_score': 5.89508, '_source': {'album_id': '7659734915', 'url': 'https://music.163.com/playlist?id=7659734915', 'name': '说唱情歌｜最怕Rapper说情话', 'author': '这条平凡的狗_QwQ', 'views': '45929', 'subscription': '286', 'label': '华语|流行|说唱', 'description': '介绍：\n恋爱中的很多人的忠诚都是演的，所以他结婚了\n\n我永远不会辜负你，因为我要把你娶回家\n\n你说你想要浪漫，那我就拼了命给你制造\n', 'amount': '69', 'nameofmusic': '着迷|所以你睡了没|粉色苏打|Darling|i adore u|靠近一点|噩梦|宝贝我不渣|丘比特|湖'}, 'highlight': {'name': ['<em>说</em><em>唱</em>情歌｜最怕Rapper<em>说</em>情话']}}, {'_index': 'index_music', '_type': '_doc', '_id': '77', '_score': 5.7539387, '_source': {'album_id': '7651196084', 'url': 'https://music.163.com/playlist?id=7651196084', 'name': '说唱派对 | emo走心励志伤感旋律中文说唱', 'author': '芳心收集人士', 'views': '235421', 'subscription': '1888', 'label': '华语|说唱|

C:\Users\BR\AppData\Local\Temp/ipykernel_12284/2869991905.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="index_music", body=query)


In [171]:
cnt = es.count()
print((cnt))

{'count': 10023, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [61]:
def query_by_name(name):
    query_name = {
        "query": {
        "match": {
          "name": name
        }
      },
        "highlight":{
            "fields":{
                "name":{}
            }
        }
    }
    result = es.search(index="index_music", body=query_name)
    print(result)


In [62]:
def query_by_author(author):
    query_title = {
        "query": {
        "match": {
          "authors": author
        }
      },
        "highlight":{
            "fields":{
                "authors":{}
            }
        }
    }
    result = es.search(index="index_music", body=query_title)
    print(result)

In [63]:
def query_by_label(label):
    query_label = {
        "query": {
        "match": {
          "label": label
        }
      },
        "highlight":{
            "fields":{
                "label":{}
            }
        }
    }
    result = es.search(index="index_music", body=query_label)
    print(result)

In [64]:
def query_by_nameofmusic(nameofmusic):
    query_nameofmusic = {
        "query": {
        "match": {
          "nameofmusic": nameofmusic
        }
      },
        "highlight":{
            "fields":{
                "nameofmusic":{}
            }
        }
    }
    result = es.search(index="index_music", body=query_nameofmusic)
    print(result)

In [67]:
if __name__ == '__main__':
    method = input("please select the type you want to search for: 1. name 2. author 3. label 4. name of music")
    if method=="1":
        name = input("Please enter the name")
        query_by_name(name)
    elif method=="2":
        author = input("Please enter the author")
        query_by_author(author)
    elif method=="3":
        label = input("Please enter the label")
        query_by_label(label)
    elif method=="4":
        nameofmusic = input("Please enter the name of music")
        query_by_nameofmusic(nameofmusic)


{'took': 13, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 92, 'relation': 'eq'}, 'max_score': 19.713055, 'hits': [{'_index': 'index_music', '_type': '_doc', '_id': '3', '_score': 19.713055, '_source': {'album_id': '2235972598', 'url': 'https://music.163.com/playlist?id=2235972598', 'name': '『经典华语3000首』还是老歌有味道', 'author': '音乐想象家', 'views': '3761257', 'subscription': '26213', 'label': '华语|粤语|经典', 'description': '介绍：\n总是在老歌里，我才会充分意识到自己的温情。流金岁月，配上简单的老歌，也便配上了真实的悲欢。\n', 'amount': '3440', 'nameofmusic': '粉红色的回忆|我|爱|你|爱|我|当|问|我|问'}, 'highlight': {'nameofmusic': ['<em>粉</em><em>红</em><em>色</em><em>的</em><em>回</em><em>忆</em>|我|爱|你|爱|我|当|问|我|问']}}, {'_index': 'index_music', '_type': '_doc', '_id': '45', '_score': 8.032837, '_source': {'album_id': '6994605030', 'url': 'https://music.163.com/playlist?id=6994605030', 'name': '罗翔励志哲理.', 'author': '那年盛夏的ta', 'views': '471864', 'subscription': '6635', 'label': '流行|感动|华语', 'descript

C:\Users\BR\AppData\Local\Temp/ipykernel_12284/1826592748.py:14: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="index_music", body=query_nameofmusic)
